# Procesamiento de Imágenes con MoE - Análisis Detallado

Este notebook proporciona un análisis profundo del procesamiento de imágenes usando la arquitectura Mixture of Experts (MoE).

In [ ]:
import sys
sys.path.append('..')

from image_processing.image_moe import ImageMoE
from utils.dashboard import ComplexityDashboard
import numpy as np
import matplotlib.pyplot as plt
from rich.console import Console

console = Console()

## 1. Generación de Patrones de Prueba

Creamos diferentes patrones de imagen para probar la capacidad de especialización de los expertos.

In [ ]:
def generate_test_patterns(size=32):
    patterns = {
        'uniform': np.ones((size, size)) * 0.5,  # Patrón uniforme
        'gradient': np.linspace(0, 1, size).reshape(-1, 1) * np.ones((1, size)),  # Gradiente
        'checkerboard': np.indices((size, size)).sum(axis=0) % 2,  # Tablero de ajedrez
        'noise': np.random.rand(size, size),  # Ruido aleatorio
        'circle': np.fromfunction(
            lambda i, j: ((i-size/2)**2 + (j-size/2)**2 < (size/3)**2),
            (size, size)
        ).astype(float)  # Círculo
    }
    return patterns

# Generar y visualizar patrones
patterns = generate_test_patterns()
fig, axes = plt.subplots(1, len(patterns), figsize=(20, 4))

for ax, (name, pattern) in zip(axes, patterns.items()):
    ax.imshow(pattern, cmap='gray')
    ax.set_title(name.capitalize())
    ax.axis('off')

plt.tight_layout()
plt.show()

## 2. Análisis de Complejidad Regional

Analizamos cómo el MoE procesa diferentes regiones de las imágenes.

In [ ]:
def analyze_pattern(pattern_name, image_data, moe):
    print(f"\nAnalizando patrón: {pattern_name}")
    
    # Procesar imagen
    result = moe.process(image_data)
    
    # Crear mapa de expertos
    expert_map = np.zeros_like(image_data)
    for i in range(0, image_data.shape[0], 8):
        for j in range(0, image_data.shape[1], 8):
            region = image_data[i:min(i+8, image_data.shape[0]),
                               j:min(j+8, image_data.shape[1])]
            features = moe._get_region_features(region)
            expert_weights = moe._get_expert_weights(features)
            expert = np.argmax(expert_weights)
            expert_map[i:i+8, j:j+8] = expert
    
    # Visualizar resultados
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    ax1.imshow(image_data, cmap='gray')
    ax1.set_title('Imagen Original')
    ax1.axis('off')
    
    im = ax2.imshow(expert_map, cmap='viridis')
    ax2.set_title('Asignación de Expertos')
    ax2.axis('off')
    plt.colorbar(im, ax=ax2)
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar estadísticas
    unique, counts = np.unique(expert_map, return_counts=True)
    total = np.prod(expert_map.shape)
    
    print("\nDistribución de expertos:")
    for expert, count in zip(unique, counts):
        percentage = (count / total) * 100
        print(f"Expert {int(expert)}: {count} píxeles ({percentage:.1f}%)")

# Analizar cada patrón
moe = ImageMoE(num_experts=4)
for name, pattern in patterns.items():
    analyze_pattern(name, pattern, moe)

## 3. Métricas de Rendimiento

Evaluamos el rendimiento del MoE en términos de tiempo de procesamiento y confianza.

In [ ]:
import time
def evaluate_performance(patterns, num_iterations=10):
    processing_times = []
    complexities = []
    
    for _ in range(num_iterations):
        for name, pattern in patterns.items():
            start_time = time.perf_counter()
            moe.process(pattern)
            end_time = time.perf_counter()
            
            processing_times.append(end_time - start_time)
            complexity = moe._compute_complexity(pattern)
            complexities.append(complexity)
    
    # Visualizar resultados
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Tiempos de procesamiento
    ax1.hist(np.array(processing_times) * 1000, bins=20)
    ax1.set_title('Distribución de Tiempos de Procesamiento')
    ax1.set_xlabel('Tiempo (ms)')
    ax1.set_ylabel('Frecuencia')
    
    # Complejidad
    ax2.hist(complexities, bins=20)
    ax2.set_title('Distribución de Complejidad')
    ax2.set_xlabel('Complejidad')
    ax2.set_ylabel('Frecuencia')
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar estadísticas
    print("\nEstadísticas de rendimiento:")
    print(f"Tiempo promedio: {np.mean(processing_times)*1000:.2f} ms")
    print(f"Desviación estándar: {np.std(processing_times)*1000:.2f} ms")
    print(f"Complejidad promedio: {np.mean(complexities):.2f}")

evaluate_performance(patterns)

## 4. Conclusiones

Este análisis demuestra cómo el MoE:
1. Asigna diferentes expertos según las características de la región
2. Maneja eficientemente diferentes tipos de patrones
3. Mantiene un rendimiento consistente en términos de tiempo de procesamiento

Los expertos se especializan en:
- Regiones uniformes oscuras
- Regiones uniformes brillantes
- Detección de bordes
- Análisis de texturas complejas